In [1]:
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from oauth2client import file , client , tools 
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.http import MediaFileUpload , MediaIoBaseUpload
from airflow.models import Variable

import os
import re
import datetime
import json
import requests
import socket
import zipfile
socket.setdefaulttimeout(150)

## Airflow Variable

In [2]:
PATH_DATABASE_BACKUP = Variable.get('PATH_DATABASE_BACKUP')
PATH_GOOGLEDRIVE_CREDS = Variable.get('PATH_GOOGLEDRIVE_CREDS')
PATH_GOOGLEDRIVE_TOKEN = Variable.get('PATH_GOOGLEDRIVE_TOKEN')

## Self Definited Variable

In [3]:
# backup file variables
BACKUP_DATE = str(datetime.date.today()).replace('-','')
DATABASE_NAME = 'postgres'
BACKUP_USER = 'stockbuneo'
BACKUP_FILE_NAME = 'DB_Stock'

# upload google drive variables
TARGET_FOLDER = ['DB_STOCK_BAK']

PAGESIZE = lambda x : 10 if x == None else len(x)

In [4]:
def CREATE_BACKUP_FILE( PATH_DATABASE_BACKUP , BACKUP_DATE , DATABASE_NAME , BACKUP_USER , BACKUP_FILE_NAME ):
    
    FILE_NAME = f'{BACKUP_FILE_NAME}_{BACKUP_DATE}'
    #Create backup file
    os.system(f"""pg_dump {DATABASE_NAME} -U {BACKUP_USER} -f {PATH_DATABASE_BACKUP}/{FILE_NAME}.sql""")
    
    #Add to Zipfile
    zf = zipfile.ZipFile(f'{PATH_DATABASE_BACKUP}/{FILE_NAME}.zip' , 'w' , zipfile.ZIP_DEFLATED)
    zf.write(f'{PATH_DATABASE_BACKUP}/{FILE_NAME}.sql')
    
    #Delete sql type of backup file
    os.remove(f'{PATH_DATABASE_BACKUP}/{FILE_NAME}.sql')
    
    print(f'Backup File Finished , Path : {PATH_DATABASE_BACKUP}/{BACKUP_FILE_NAME}_{BACKUP_DATE}.zip')
    return FILE_NAME

In [5]:
def DELETE_PAST_BACKUPFILE(PATH_DATABASE_BACKUP) :
    
    CLEAN_DIRLIST = []
    BAKFOLDER_PRENAME = ''    
    
    DIRLIST = os.listdir(PATH_DATABASE_BACKUP)
    
    for dir_name in DIRLIST:
        CLEAN_DIRLIST.append(dir_name[-12:])
        BAKFOLDER_PRENAME = dir_name[:-12]
    
    CLEAN_DIRLIST.sort(reverse = True)
    CLEAN_DIRLIST = CLEAN_DIRLIST[2:]
    
    print('Delet Files List\n---------------------------------')
    
    for del_date in CLEAN_DIRLIST:
        os.remove(f'{PATH_DATABASE_BACKUP}/{BAKFOLDER_PRENAME}{del_date}')
        print(f'{PATH_DATABASE_BACKUP}/{BAKFOLDER_PRENAME}{del_date}')
        
    print('---------------------------------')
    print(f'Delete {len(CLEAN_DIRLIST)} Expired Files')

In [6]:
def CONNECT_SERVICE( PATH_GOOGLEDRIVE_CREDS , PATH_GOOGLEDRIVE_TOKEN ):
    
    SCOPES = 'https://www.googleapis.com/auth/drive'   
    creds = None    
    
    if os.path.exists(PATH_GOOGLEDRIVE_TOKEN):
        creds = Credentials.from_authorized_user_file(PATH_GOOGLEDRIVE_TOKEN)
        
    if not creds or not creds.valid:
        
        if (creds and creds.expired and creds.refresh_token) :
            creds.refresh(Request())

        else :
            flow = InstalledAppFlow.from_client_secrets_file(PATH_GOOGLEDRIVE_CREDS, SCOPES)
            creds = flow.run_local_server(port=0)
            
            token = open(PATH_GOOGLEDRIVE_TOKEN,'w')
            token.write(creds.to_json())
            
    service = build('drive', 'v3',credentials = creds )
    
    return service

In [7]:
def GET_PARENTID(service , TARGET_FOLDER = None):
    
    PAGESIZE = lambda x : 10 if x == None else len(x)
    
    q = "("
    for n , target in enumerate(TARGET_FOLDER):
        if n == 0:
            text = f"name = '{target}'"
            q = q + text
        else :
            text = f" or name = '{target}'"
            q = q + text
    q = q + ") and trashed = False "   

    results = service.files().list(pageSize = PAGESIZE(TARGET_FOLDER) , fields = "nextPageToken, files(id,name)" ,
                                   q = q).execute()

    items = results.get('files', [])
    if not items:     
        print('No files found.') 

    return items

In [10]:
def UPLOAD_FILE(service , items , PATH_DATABASE_BACKUP , FILE_NAME):   
       
        full_path = f'{PATH_DATABASE_BACKUP}/{FILE_NAME}.zip'

        print(full_path)
        mime = 'application/zip'
        file_matadata = {'name' : f'{BACKUP_FILE_NAME}_{BACKUP_DATE}.zip' , 
                         'parents' : items[0]['id'] }
        
        media = MediaFileUpload(full_path,)
        
        service.files().create(body = file_matadata , media_body = media ,fields = 'id' ).execute()
        print("File Upload Success")

In [11]:
if __name__ == '__main__':
    FILE_NAME = CREATE_BACKUP_FILE( PATH_DATABASE_BACKUP , BACKUP_DATE , DATABASE_NAME , BACKUP_USER , BACKUP_FILE_NAME )
    DELETE_PAST_BACKUPFILE(PATH_DATABASE_BACKUP)
    service = CONNECT_SERVICE(PATH_GOOGLEDRIVE_CREDS , PATH_GOOGLEDRIVE_TOKEN )
    items = GET_PARENTID(service , TARGET_FOLDER = TARGET_FOLDER)
    UPLOAD_FILE(service , items , PATH_DATABASE_BACKUP , FILE_NAME)

[2021-11-07 21:14:44,265] {__init__.py:49} INFO - file_cache is only supported with oauth2client<4.0.0
/home/buneo/Stock/Database_backup/DB_Stock_20211107.zip
File Upload Success
